# Labo 3 - ARN

In [ ]:
import pandas as pd

mice_eeg = pd.read_csv('./pw3_data/EEG_mouse_data_1.csv')
mice_eeg2 = pd.read_csv('./pw3_data/EEG_mouse_data_2.csv')
df = pd.concat([mice_eeg, mice_eeg2], ignore_index=True)

mice_eeg3 = pd.read_csv('./pw3_data/EEG_mouse_data_test.csv')
df

## 1. Introduction

We take the first 25 features as they are contain the biggest occurences count.

In [ ]:
df = df.iloc[:, 1:26]
df

Normalisation


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Normalize
dfn = scaler.fit(df)

# Shuffle
# df.sample(frac=1) #TODO: obsolete because of KFold ?

# Cut in 3 folds (1/3 for test and 2/3 for training)
# shuffle d'abord avant de découper
#  normaliser tout fit
# partager

import keras
from keras import layers
from sklearn.model_selection import KFold
import matplotlib.pyplot as pl


pl.clf()

keras.utils.set_random_seed(123)
kf = KFold(n_splits=3, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(df)):
    print(str(i) + ' ' + str(train_index) + ' ' + str(test_index))
# See training 2 cells below

In [ ]:
# Configure and setup our MLP
# Source: From the notebook 5. See 'Changed: '
def create_model():
  mlp = keras.Sequential([
      layers.Input(shape=(25,)), # Changed: shape=(25,) instead of 2
      layers.Dense(2, activation="tanh"), # Try different numbers of hidden neurons here (e.g. 2, 4, 8, 32, 128)
      # Changed: todo try to change the amount of layers and perceptrons per layer
      layers.Dense(1, activation="tanh"),   # next 
  ])

  # Experiment with hyperparameters here:
  # momentum: [0, 0.8, 0.9, 0.99]
  # learning_rate: [0.1, 0.01, 0.001, 0.0001]
  mlp.compile(
      optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.99),
      loss="mse",
  )

  return mlp

mlp = create_model()
mlp.summary()



In [ ]:
df

In [ ]:
# Run training on 3 folds
history_list = []
trained_mlp = []

for i, (train_index, test_index) in enumerate(kf.split(df)):
  print(str(i) + ' ' + str(train_index) + ' ' + str(test_index))
  # We need to create a new model everytime otherwise fit will continue previous training
  mlp = create_model()

  history = mlp.fit(
      x=input_data[train_index], y=output_data[train_index],
      validation_data=(df[test_index], output_data[test_index]),
      epochs=400
  )

  history_list.append(history)
  trained_mlp.append(mlp)

# TODO: fix this bug with input data and output data, how to use them ??

In [ ]:

# Note for later: next section will use 3 instead of 1 because 3 classes, and ~ supermax 
layers.Dense(1, activation="tanh"),   # next 
